In [ ]:
from sampling.images import plot_frame_count_distributions

samples_root_dir = "../data/samples"

plot_frame_count_distributions(samples_root_dir)


In [ ]:
from sota.model import train_fresh_model

train_fresh_model("../data")

In [ ]:
from sota.model import train_best_model

train_best_model("../data", optimizer="AdamW", lr0=0.0005)

In [ ]:
from sota.model import get_fresh_model

model = get_fresh_model()

metrics = model.val(data="data/img/techniques/val")

In [1]:
from hpe_dnn.model import train_fresh_model

model = train_fresh_model("../data")

2025-05-03 13:48:30.721601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746272911.644744   76186 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746272911.891690   76186 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-03 13:48:34.239394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


     NOSE_x    NOSE_y    NOSE_z  NOSE_visibility  LEFT_SHOULDER_x  \
0  0.513114  0.283191  0.046420         0.999967         0.478404   
1  0.522152  0.313473  0.044570         0.999968         0.487691   
2  0.351092  0.391248 -0.002703         0.998592         0.366234   
3  0.370027  0.297158  0.240357         0.998023         0.337771   
4  0.379843  0.517894 -0.599730         0.998220         0.400984   

   LEFT_SHOULDER_y  LEFT_SHOULDER_z  LEFT_SHOULDER_visibility  LEFT_ELBOW_x  \
0         0.339587        -0.101192                  0.999968      0.423005   
1         0.362177        -0.112082                  0.999967      0.426185   
2         0.438087        -0.142772                  0.999928      0.301032   
3         0.365539         0.185555                  0.999909      0.294720   
4         0.591487        -0.377551                  0.999916      0.326186   

   LEFT_ELBOW_y  ...  LEFT_THUMB_TIP_x  LEFT_THUMB_TIP_y  LEFT_THUMB_TIP_z  \
0      0.327103  ...            

/mnt/c/Projects/climbing-technique-detector/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
I0000 00:00:1746272957.231817   76186 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22276 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:21:00.0, compute capability: 7.5
2025-05-03 13:49:22.479051: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-05-03 13:49:22.858031: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-05-03 13:49:23.588316: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-05-03 13:49:25.057848: I tensorflow

Epoch 1/10


I0000 00:00:1746273004.340973   76640 service.cc:148] XLA service 0x7f872403ecc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746273004.341367   76640 service.cc:156]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2025-05-03 13:50:04.628692: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746273005.333803   76640 cuda_dnn.cc:529] Loaded cuDNN version 90300


 12/121 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1814 - loss: 0.6951

I0000 00:00:1746273010.439519   76640 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5366 - loss: 0.4522
Epoch 1: val_accuracy improved from -inf to 0.68689, saving model to ../data/runs/hpe_dnn/train1/models/epoch_01__val_accuracy_0.6869.keras
121/121 ━━━━━━━━━━━━━━━━━━━━ 20s 96ms/step - accuracy: 0.5372 - loss: 0.4514 - val_accuracy: 0.6869 - val_loss: 0.2511
Epoch 2/10
120/121 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6587 - loss: 0.2604
Epoch 2: val_accuracy improved from 0.68689 to 0.71238, saving model to ../data/runs/hpe_dnn/train1/models/epoch_02__val_accuracy_0.7124.keras
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.6588 - loss: 0.2603 - val_accuracy: 0.7124 - val_loss: 0.2103
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6926 - loss: 0.2257
Epoch 3: val_accuracy improved from 0.71238 to 0.74879, saving model to ../data/runs/hpe_dnn/train1/models/epoch_03__val_accuracy_0.7488.keras
121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6926 - loss: 0.2257 - val_accur

In [ ]:
from keras._tf_keras.keras.models import load_model


In [ ]:
model.summary()

In [ ]:
results = model.evaluate(val_ds)

In [ ]:
from datetime import datetime
print(datetime.now())

In [ ]:
from os.path import join
model_name = f"arch1__{datetime.now()}__val_acc__0.8240.keras"
model_path = join("../data", "runs", "hpe_dnn", "model", model_name)
model.save(model_path)